In [ ]:
from selenium import webdriver  # powers the browser interaction
from selenium.webdriver.support.ui import Select  # selects menu options
from bs4 import BeautifulSoup  # to parse HTML
import csv  # to write CSV
import pandas as pd  # to see CSV
import time
import os
import random
import requests
from string import punctuation
import re
import urllib

In [ ]:
df = pd.read_csv('data/AHA-jobs.csv')

In [ ]:
def get_key(x):
    to_replace = ['Medieval Institute','History', 'Department', 'Divinity', 'School', '  ', 'Public Policy', 'Business', 'Center', 'Studies']
    
    for r in to_replace:
        x = x.replace(r, '')
        
#     more = x.split(',')
#     if len(more) > 1:
#         for i in more:
#             if 'College' in i:
#                 x = i
#             elif 'University' in i:
#                 x = i
    x = x.strip()      
    if x[-2:] == 'of':
        x = x[:-2]
        
    for p in punctuation:
        x = x.replace(p, '')
                
    return x.strip()

In [ ]:
all_dicts = []
driver = webdriver.PhantomJS()
for search in set(df['employer']):

    print(search)
    driver.get("https://www.wikipedia.org/")

    driver.find_element_by_css_selector('#searchInput').send_keys(get_key(search))

    driver.find_element_by_css_selector('#search-form > fieldset > button > i').click()

    soup = BeautifulSoup(driver.page_source, 'html5lib')
    
    uni_dict = {}
    uni_dict['search'] = search
    
    try:

        if soup.find('h1', {'id': 'firstHeading'}).text.strip() == 'Search results':
            base = 'https://en.wikipedia.org'
            extension = soup.find_all('div', {'class': 'mw-search-result-heading'})[0].find('a')['href']
            driver.get(base + extension)


        detail_rows = soup.find('table', {'class': 'infobox vcard'}).find_all('tr')

        for d in detail_rows:
            if d.find('th') != None:
                key = d.find('th').text.strip()
                value = d.find('td').text.strip()
                uni_dict[key] = value
    
    except:
        pass
    
    all_dicts.append(uni_dict)

    
    time.sleep(1)

In [ ]:
all_dicts[2]

In [ ]:
import json

json.dump(all_dicts, open('wiki.json', 'w'))

In [ ]:
def get_info(x, key):
    for d in all_dicts:
        if d['search'] == x:
            if key in d:
                return d[key]
            else:
                return None

In [ ]:
df['postgraduates'] = df['employer'].apply(get_info, args=('Postgraduates',))
df['undergraduates'] = df['employer'].apply(get_info, args=('Undergraduates',))
df['endowment'] = df['employer'].apply(get_info, args=('Endowment',))
df['established'] = df['employer'].apply(get_info, args=('Established',))
df['campus'] = df['employer'].apply(get_info, args=('Campus',))
df['academic_staff'] = df['employer'].apply(get_info, args=('Academic staff',))

In [ ]:
df.head()

In [ ]:
def graduates(x):
    if type(x) == str:
        match = re.search(r'([0-9]+,?[0-9]+)', x)
        if match.groups():
            return match.groups()[0].replace(',', '')
    else:
        return None

In [ ]:
def acres(x):
    if type(x) == str:
        match = re.search(r'([0-9]+,?[0-9]+)\sacres', x)
        if match != None:
            return match.groups()[0].replace(',', '')
    else:
        return None

In [ ]:
def year(x):
    if type(x) == str:
        match = re.search(r'([0-9]{4})', x)
        if match != None:
            return match.groups()[0]
    else:
        return None

In [ ]:
def endowment(x):
    if type(x) == str:
        match = re.search(r'(\$([0-9]+\.?[0-9]+)\s(million|billion))', x)
        if match != None:
            num = float(match.groups()[1])
            if match.groups()[2] == 'billion':
                num = num * 1000000000
                return str(num)[:-2]
            elif match.groups()[2] == 'million':
                num = num * 1000000
                return str(num)[:-2]

In [ ]:
df['postgraduates'] = df['postgraduates'].apply(graduates)
df['undergraduates'] = df['undergraduates'].apply(graduates)
df['endowment'] = df['endowment'].apply(endowment)
df['established'] = df['established'].apply(year)
df['campus'] = df['campus'].apply(acres)
df['academic_staff'] = df['academic_staff'].apply(graduates)

In [ ]:
df.head()

In [ ]:
df.to_csv('testing.csv', index=False)

In [ ]:
df2 = pd.read_csv('testing.csv')

In [ ]:
df2.describe()

In [ ]:
df2.corr()

In [ ]:
%matplotlib inline

In [ ]:
df2.plot.scatter('undergraduates', 'established')

In [ ]:
df2.plot.scatter('postgraduates', 'academic_staff')

In [ ]:
latitude = []
longitude = []
for i in df2['employer']:
    search = urllib.parse.quote(i)
    
    print(search)

    try:
        json_res = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(search)).json()
        coordinates = json_res['results'][0]['geometry']['location']
        latitude.append(coordinates['lat'])
        longitude.append(coordinates['lng'])
    except:
        latitude.append(None)
        longitude.append(None)

    time.sleep(.5)

In [ ]:
df2['latitude'] = latitude
df2['longitude'] = longitude

In [ ]:
df2.head()

In [ ]:
df2.to_csv('test2.csv', index=False)

In [ ]:
pd.read_csv('test2.csv').describe()